This Version looks up the Publication Spreadsheet to pull data direct from intake form

In [69]:
# Install packages as needed
#%pip install openpyxl
#%pip install pandas 
#%pip install plotly
#%pip install nbformat
#%pip install --upgrade ipython

In [70]:
#import libraries
import pandas as pd
import json
import os
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import locale
import numpy as np
import re

Extract

In [71]:
###EXTRACT###
# Read Transaction Excel data into a Pandas dataframe
file = r'C:\Users\DIM6\OneDrive - PGE\[Work] [Shared]\CXI Projects\20200122 03 CC Fact Sheet\2022\2022 Customer Transactions Map - Publication Sheets.xlsx'
#file = r'C:\Users\DIM6\OneDrive - PGE\Documents\GitHub\davemccallme.github.io\Cust-Trxn-Channel-Charts\Trxn-Data-Sunburst.xlsx' #uses rough trxn data
sheet_name = 'Sunburst'

# Read the specific sheet from the Excel file
df = pd.read_excel(file, sheet_name=sheet_name)

df['Value'] = pd.to_numeric(df['Value'])

print(df.head())

  Root Level Level 1 Level 2 L3 Classifier  Value  TableBlank?  Level  \
0        NaN     NaN     NaN           NaN    NaN        False    NaN   
1        NaN     NaN     NaN           NaN    NaN        False    NaN   
2        NaN     NaN     NaN           NaN    NaN        False    NaN   
3        NaN     NaN     NaN           NaN    NaN        False    NaN   
4        NaN     NaN     NaN           NaN    NaN        False    NaN   

   Unnamed: 7                        0 0.1 59.844395  
0         NaN   % Payment transactions   0      2022  
1         NaN              Pay by Mail   0  0.187152  
2         NaN                      Web   0  0.467045  
3         NaN  Other Elec Pay Channels   0  0.243905  
4         NaN              Call Center   0  0.072321  


In [72]:
#Column Headers
Root = 'Root Level'
L1 = "Level 1" 
L2 = "Level 2"
Value = "Value"
Cat_Level ="Level"

In [73]:
 #Remove columns not needed for sunburst
df = df[[Root, L1, L2, Value, Cat_Level]]

print(df.head())

  Root Level Level 1 Level 2  Value  Level
0        NaN     NaN     NaN    NaN    NaN
1        NaN     NaN     NaN    NaN    NaN
2        NaN     NaN     NaN    NaN    NaN
3        NaN     NaN     NaN    NaN    NaN
4        NaN     NaN     NaN    NaN    NaN


In [74]:
# Remove rows with all NaN values
df = df.dropna(how='all')

# Filter out rows where "Level" is 0
df = df[df[Cat_Level] != 0]

# Filter out rows with NaN or blank values in the "Value" column
df = df[(df[Value].notna()) & (df[Value] != '')]

# multiply the dataframe by 1 million to return to nominal units (from Million to 0)
df[Value] = df[Value].mul(1000000)

## Replace NaN with None for plotly sunburst chart
df = df.where(pd.notna(df), "")

print(df.head())

          Root Level   Level 1 Level 2       Value  Level
43               NPC  Payments           1770000.0    1.0
49              Mail  Payments          11200000.0    1.0
58  Other Electronic  Payments          10393000.0    1.0
59  Other Electronic  Payments           2735000.0    1.0
60  Other Electronic  Payments            637000.0    1.0


In [75]:
def fill_empty_L2(row):
    if row[L2] == "":
        return row[L1]
    else:
        return row[L2]

# Fill empty strings in L2 with corresponding L1 values
#df[L2] = df.apply(fill_empty_L2, axis=1)

In [76]:
# Remove "-" and whitespace
df[L1] = df[L1].str.replace(r"\s*-\s*", "").str.strip().astype(str)
df[L2] = df[L2].str.replace(r"\s*-\s*", "").str.strip().astype(str)

#Convert L2 to proper case
df[L2] = df[L2].str.title()

C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_16612\1351419079.py:2: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_16612\1351419079.py:3: FutureWarning:

The default value of regex will change from True to False in a future version.



In [77]:
# Splice a new df
df = df.copy()

print(df.head())

          Root Level   Level 1 Level 2       Value  Level
43               NPC  Payments           1770000.0    1.0
49              Mail  Payments          11200000.0    1.0
58  Other Electronic  Payments          10393000.0    1.0
59  Other Electronic  Payments           2735000.0    1.0
60  Other Electronic  Payments            637000.0    1.0


In [78]:
# Convert dataframe to a hierarchical dictionary
def make_hierarchy_dict(df, path=[], value_col=Value, threshold=10):
    if len(path) == df.columns.nlevels - 1:
        total = int(df[value_col].sum())
        if total < threshold:
            return None
        else:
            return total
    else:
        level = df.columns[len(path)]
        subdict = {}
        for key in df[level].dropna().unique():
            subdf = df[df[level] == key].drop(columns=[level])
            subhierarchy_dict = make_hierarchy_dict(subdf, path + [key], value_col, threshold)
            if subhierarchy_dict is not None:
                subdict[key] = subhierarchy_dict
        if len(subdict) == 0:
            return None
        else:
            return subdict

hierarchy_dict = make_hierarchy_dict(df.set_index(list(df.columns[:3])), value_col=Value, threshold=10)

print(hierarchy_dict)

110216274


In [79]:
# Convert hierarchical dictionary to Plotly-compatible JSON
def make_plotly_sunburst(hierarchy_dict, name=""):
    if isinstance(hierarchy_dict, dict):
        children = []
        for key, value in hierarchy_dict.items():
            children.append(make_plotly_sunburst(value, key))
        return {
            "name": name,
            "children": children
        }
    else:
        return {
            "label": name,
            "value": hierarchy_dict
        }

In [80]:
# Write JSON data to a file
plotly_data = make_plotly_sunburst(hierarchy_dict)

with open("plotly_data.json", "w") as f:
    json.dump(plotly_data, f)

    print(plotly_data)

{'label': '', 'value': 110216274}


In [81]:
# Write dataframe to an Excel file for QC
with pd.ExcelWriter('Sunburstdataframe copy.xlsx') as writer:
    df.to_excel(writer, sheet_name='DataFrame', index=False)

In [82]:
 #Create a dictionary to map the Root Level to its corresponding color
color_map = {'Web': 'orange', 'Other Electronic': 'gray', 'Mail': 'burlywood', 'Call Center - IVR': 'deepskyblue','Call Center - Other':'lightblue', \
              'Call Center - Live Agent':'Turquoise', 'NPC': 'gold'}

# Format a sunburst chart using Plotly Express
color_sequence = [color_map[key] for key in color_map]
fig = px.sunburst(df, path=["Root Level", "Level 1", "Level 2"], values='Value')


# Update the layout
fig.update_layout(
    title='Transaction Channel Volume 2022',
    font=dict(size=14),
    margin=dict(t=50, l=0, r=0, b=0),
    width=1400,
    height=900,
    sunburstcolorway=color_sequence,
)
fig.show()


In [83]:
# Export the chart as an HTML file
pyo.plot(fig, filename='Trxn_sunburst_chartv2.html', auto_open=True)

'Trxn_sunburst_chartv2.html'